In [1]:
import numpy as np
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import holidays

us_holidays = holidays.UnitedStates()

# **Bond Construction**

## 1. Requirements and Inputs

In [32]:
class Treasury_Par_Bond:

    n_bonds = 0

    # ----- Instance Details -----
    def __init__(self, fv, ytm, freq, T, issue_date: date): # force issue date to be entered 
        self.fv = fv
        self.ytm = ytm
        self.freq = freq
        self.T = T

        # accept str or date for issue_date
        self.issue_date = issue_date if isinstance(issue_date, date) else date.fromisoformat(issue_date)

        # For a par bond, coupon rate = YTM
        self.coupon_rate = ytm

        # Total number of payments
        self.n_periods = self.T * self.freq # N

        # Counting the number of bonds
        Treasury_Par_Bond.n_bonds += 1

    # ----- HELPER FUNCTIONS -----
    def yield_per_coupon(self):
        return self.ytm / self.freq

    def discount_factor(self, t):
        """ 
        Don't directly use n_periods. 
        Discount factor is individualised for each payment. 
        
        """
        return 1 / ((1 + self.yield_per_coupon()) ** t)

    @staticmethod
    def is_business_day(d: date, holidays=us_holidays) -> bool:
        return d.weekday() < 5 and (not holidays or d not in holidays)

    @staticmethod
    def adjusted_business_day(d: date, holiday_calendar=us_holidays) -> date:
        """
        Roll forward to the next business day if d is weekend/holiday.
        
        """
        while d.weekday() >= 5 or d in holiday_calendar:  # 5=Sat, 6=Sun
            d += timedelta(days=1)
        
        return d

    # ----- Value of Each Coupon Payment -----
    def coupon_payment(self):
        return self.fv * self.coupon_rate / self.freq

    # ----- Cash Flow Schedule for the Bond -----
    def cash_flows(self):
        """
        Returns a list of all coupon payments,
        with the face value added to the last payment. 
        
        """
        c = self.coupon_payment()
        cash_flows = [c] * self.n_periods # N coupon payments
        cash_flows[-1] += self.fv # Nth payment = coupon + principal

        return cash_flows 

    # ----- Bond Price from YTM -----
    def bond_price_from_ytm(self):
        """
        Returns the present value (price) of the bond
        based on its yield to maturity (ytm).

        Note: US Treasuries use discrete compounding discouting
        
        """
        cash_flows = self.cash_flows()
        bond_price = sum(cf * self.discount_factor(t) for t, cf in enumerate (cash_flows, start=1)) # returns (index, value)

        return bond_price

    # ----- Cash Flows of the Bond -----
    def table_of_cash_flows(self):
        """
        Returns a list of dicts: period t, cash_flow, discount_factor, present_value.

        """
        # Setting up coupon payment date calculation 
        months_between_payments = 12 // self.freq
        scheduled_coupon_payment_date = self.issue_date 

        table = []
        for t, cf in enumerate(self.cash_flows(), start=1):
            scheduled_coupon_payment_date += relativedelta(months = months_between_payments) # Add months_between_payments to the issue_date for payment date 
            
            # adjust coupon payment date if it is a holiday 
            actual_coupon_payment_date = Treasury_Par_Bond.adjusted_business_day(scheduled_coupon_payment_date)

            df = self.discount_factor(t)

            table.append({
                "t": t,
                "scheduled_date": scheduled_coupon_payment_date,   # fixed calendar schedule
                "coupon_payment_date": actual_coupon_payment_date, # business-day adjusted
                "cash_flow": cf,
                "discount_factor": df,
                "present_value": cf * df
            })
        
        return table

    # ----- Accrued Interest, Dirty Price and Clean Price -----
    # Dirty Price = Clean Price + Accured Interest 

    @staticmethod
    def get_settlement_date_from_trade_date(trade_date, holidays=us_holidays):
        """
        US Treasury settlement: T+1 (roll forward for weekends/holidays).
        
        """
        # Base case: T+1 calendar day
        settlement_date = trade_date + timedelta(days=1)
        
        # Roll forward until business day
        while settlement_date.weekday() >= 5 or settlement_date in holidays:  # 5=Sat, 6=Sun
            settlement_date += timedelta(days=1)
        
        return settlement_date

    def accrued_days(self, trade_date, holidays=us_holidays) -> int:
        settlement_date = self.get_settlement_date_from_trade_date(trade_date, holidays)
        coupon_payment_dates = [row["coupon_payment_date"] for row in self.table_of_cash_flows()]

        if settlement_date >= coupon_payment_dates[-1]:
            return 0

        if settlement_date < coupon_payment_dates[0]:
            return max(0, (settlement_date - self.issue_date).days)

        for i in range(1, len(coupon_payment_dates)):
            if coupon_payment_dates[i-1] <= settlement_date < coupon_payment_dates[i]:
                last_coupon_date = coupon_payment_dates[i-1]
                return (settlement_date - last_coupon_date).days

        return 0

    def days_in_coupon_period(self, trade_date, holidays=us_holidays) -> int:
        """
        Actual days in the current coupon period (denominator for Actual/Actual).
        If pre-first-coupon: (first_coupon - issue_date). If post-final: 0.

        """
        settlement_date = self.get_settlement_date_from_trade_date(trade_date, holidays)
        coupon_payment_dates = [row["coupon_payment_date"] for row in self.table_of_cash_flows()]

        # After (or on) final coupon -> no period
        if settlement_date >= coupon_payment_dates[-1]:
            return 0

        # Before first coupon -> period is (first_coupon - issue_date)
        if settlement_date < coupon_payment_dates[0]:
            return (coupon_payment_dates[0] - self.issue_date).days

        # Between coupons
        for i in range(1, len(coupon_payment_dates)):
            if coupon_payment_dates[i-1] <= settlement_date < coupon_payment_dates[i]:
                last_coupon_date = coupon_payment_dates[i-1]
                next_coupon_date = coupon_payment_dates[i]
                return (next_coupon_date - last_coupon_date).days

        return 0

    def accrued_fraction(self, trade_date, holidays=us_holidays):
        """
        Actual/Actual fraction elapsed in the current coupon period.
        
        """
        denom = self.days_in_coupon_period(trade_date, holidays)
        
        if denom <= 0:
            return 0.0
        
        numer = self.accrued_days(trade_date, holidays)  

        x = numer / denom
        
        return 0.0 if x < 0 else (0.999999 if x >= 1 else x)
    

    def accrued_interest(self, trade_date, holidays=us_holidays):
        """
        Accrued = coupon per period × accrued_fraction (Actual/Actual).
        
        """
        return self.coupon_payment() * self.accrued_fraction(trade_date, holidays)

    def clean_price(self):
        """
        Quoted clean price (uses coupon-date PV).
        
        """
        return self.bond_price_from_ytm()

    def dirty_price_from_trade(self, trade_date, holidays=us_holidays) -> float:
        """
        Dirty = Clean + Accrued (accrued computed at settlement).
        
        """
        return self.clean_price() + self.accrued_interest(trade_date, holidays)

    # ----- Bond Duration -----
    def macaulay_duration(self):
        """
        Macaulay Duration (in years).
        
        """
        price = self.bond_price_from_ytm()
        weighted_sum = sum(
            t * cf * self.discount_factor(t)
            for t, cf in enumerate(self.cash_flows(), start=1)
        )
        
        return weighted_sum / price / self.freq  # divide by freq to convert periods → years

    def modified_duration(self):
        mod_duration = self.macaulay_duration() / (1 + self.yield_per_coupon())

        return mod_duration

    def pv01(self) -> float:
        """
        Price Value of 1bp (DV01).
        
        """
        return self.modified_duration() * self.clean_price() * 0.0001

    
    def convexity(self) -> float:
        """
        Convexity in years^2 (discrete compounding, standard textbook formula):
        C = [ Σ CF_t * t*(t+1) / (1+y_per)^(t+2) ] / P   (per-period units)
        Convert to years^2 by dividing by freq^2.
        
        """
        price = self.bond_price_from_ytm()
        
        if price <= 0:
            return 0.0
        
        per = self.yield_per_coupon()
        
        conv_per_period = (
            sum(
                cf * t * (t + 1) / ((1 + per) ** (t + 2))
                for t, cf in enumerate(self.cash_flows(), start=1)
            ) / price
        )
        
        return conv_per_period / (self.freq ** 2)

    def dollar_convexity(self) -> float:
        """
        Dollar convexity = price × convexity (units: currency per (Δy)^2).
        
        """
        return self.clean_price() * self.convexity()

        

In [37]:
# ---- Test Script ----

# Instantiate a 5y semiannual par Treasury (issue dated 2025-01-01)
bond_1 = Treasury_Par_Bond(1000, 0.03, 2, 5, '2025-01-01')

print("Face Value:", bond_1.fv)
print("YTM:", bond_1.ytm)
print("Coupon Frequency:", bond_1.freq)
print("Maturity (years):", bond_1.T)
print("Number of Periods:", bond_1.n_periods)

print("\nCoupon Payment per Period:", bond_1.coupon_payment())
print("Cash Flows:", bond_1.cash_flows())
print("Bond Price from YTM (Clean):", round(bond_1.bond_price_from_ytm(), 2))

print("\n--- Cash Flow Table ---")
for row in bond_1.table_of_cash_flows():
    print(
        f"t={row['t']:2d}, "
        f"Scheduled={row['scheduled_date']}, "
        f"Payment Date={row['coupon_payment_date']}, "
        f"CF=${row['cash_flow']:8.2f}, "
        f"DF={row['discount_factor']:.6f}, "
        f"PV=${row['present_value']:7.2f}"
    )

# ---- Accrued, Clean, and Dirty Prices ----
trade_date = date(2025, 6, 10)   # example trade date
settlement_date = bond_1.get_settlement_date_from_trade_date(trade_date)

print("\n--- Pricing ---")
print("Trade Date:", trade_date)
print("Settlement Date:", settlement_date)
print("Accrued Days:", bond_1.accrued_days(trade_date))
print("Days in Coupon Period:", bond_1.days_in_coupon_period(trade_date))
print("Accrued Fraction:", round(bond_1.accrued_fraction(trade_date), 6))
print("Accrued Interest:", round(bond_1.accrued_interest(trade_date), 6))
print("Clean Price:", round(bond_1.clean_price(), 6))
print("Dirty Price:", round(bond_1.dirty_price_from_trade(trade_date), 6))

# ---- Duration ----
print("\n--- Duration ---")
print("Macaulay Duration (years):", round(bond_1.macaulay_duration(), 6))
print("Modified Duration (years):", round(bond_1.modified_duration(), 6))
print("PV01: ", round(bond_1.pv01(), 6))

# ---- Convexity ----
print("\n--- Convexity ---")
print("Convexity: ", round(bond_1.convexity(), 6))
print("Dollar Convexity: ", round(bond_1.dollar_convexity(), 6))

Face Value: 1000
YTM: 0.03
Coupon Frequency: 2
Maturity (years): 5
Number of Periods: 10

Coupon Payment per Period: 15.0
Cash Flows: [15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0, 1015.0]
Bond Price from YTM (Clean): 1000.0

--- Cash Flow Table ---
t= 1, Scheduled=2025-07-01, Payment Date=2025-07-01, CF=$   15.00, DF=0.985222, PV=$  14.78
t= 2, Scheduled=2026-01-01, Payment Date=2026-01-02, CF=$   15.00, DF=0.970662, PV=$  14.56
t= 3, Scheduled=2026-07-01, Payment Date=2026-07-01, CF=$   15.00, DF=0.956317, PV=$  14.34
t= 4, Scheduled=2027-01-01, Payment Date=2027-01-04, CF=$   15.00, DF=0.942184, PV=$  14.13
t= 5, Scheduled=2027-07-01, Payment Date=2027-07-01, CF=$   15.00, DF=0.928260, PV=$  13.92
t= 6, Scheduled=2028-01-01, Payment Date=2028-01-03, CF=$   15.00, DF=0.914542, PV=$  13.72
t= 7, Scheduled=2028-07-01, Payment Date=2028-07-03, CF=$   15.00, DF=0.901027, PV=$  13.52
t= 8, Scheduled=2029-01-01, Payment Date=2029-01-02, CF=$   15.00, DF=0.887711, PV=$  13.32
t= 9, 